In [15]:
import sqlite3
import pandas as pd
import csv
from typing import List
import re
from astroquery.simbad import Simbad


In [135]:
def create_database_schema(conn: sqlite3.Connection):
    cursor = conn.cursor()
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS system (
        sys_name VARCHAR(100) PRIMARY KEY,
        sy_snum INTEGER,
        sy_pnum INTEGER,
        ra FLOAT,
        dec FLOAT
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS discovery (
        disc_refname VARCHAR(200) PRIMARY KEY,
        disc_pubdate DATE
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS molecule (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(100) UNIQUE
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS planet (
        pl_name VARCHAR(100) PRIMARY KEY,
        hostname VARCHAR(100),
        mass FLOAT,
        radius FLOAT,
        orbital_period FLOAT,
        tsm FLOAT,
        disc VARCHAR(200),
        FOREIGN KEY (hostname) REFERENCES system(sys_name),
        FOREIGN KEY (disc) REFERENCES discovery(disc_refname)
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS planet_has_molecule (
        pl_name VARCHAR(100),
        molecule_id INTEGER,
        FOREIGN KEY (pl_name) REFERENCES planet(pl_name),
        FOREIGN KEY (molecule_id) REFERENCES molecule(id),
        PRIMARY KEY (pl_name, molecule_id)
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS instrument (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name VARCHAR(100) UNIQUE
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS planet_characterized_by (
        pl_name VARCHAR(100),
        instrument_id INTEGER,
        FOREIGN KEY (pl_name) REFERENCES planet(pl_name),
        FOREIGN KEY (instrument_id) REFERENCES instrument(id),
        PRIMARY KEY (pl_name, instrument_id)
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS host_ids (
        primary_id VARCHAR(100),
        alternate_id VARCHAR(100),
        FOREIGN KEY (primary_id) REFERENCES system(sys_name)
        PRIMARY KEY (primary_id, alternate_id)

    )
    ''')    


    cursor.execute('''
    CREATE TABLE IF NOT EXISTS brewer_stellar_property (
        StellarID TEXT PRIMARY KEY,
        Teff FLOAT,
        log_g FLOAT,
        SN FLOAT,
        Mass FLOAT,
        l_Mass FLOAT,
        u_Mass FLOAT,
        Age FLOAT,
        l_Age FLOAT,
        u_Age FLOAT,
        M_H FLOAT,
        e_M_H FLOAT
    )
    ''')

    cursor.execute('''
    CREATE TABLE IF NOT EXISTS apogee_stellar_property (
        StellarID TEXT PRIMARY KEY,
        Teff FLOAT,
        e_Teff FLOAT,
        log_g FLOAT,
        e_log_g FLOAT,
        SN FLOAT,
        M_H FLOAT,
        e_M_H FLOAT
    )
    ''')
    
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS abundance_surveys (
        SurveyID INTEGER PRIMARY KEY AUTOINCREMENT,
        Name VARCHAR(100) UNIQUE
    )
    ''')
    
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS elemental_abundances (
        StellarID VARCHAR(100) NOT NULL,
        Element_Ratio VARCHAR(20) NOT NULL,
        Abundance FLOAT,
        Error FLOAT,
        SurveyID INTEGER,
        FOREIGN KEY (StellarID) REFERENCES brewer_stellar_property(StellarID),
        FOREIGN KEY (SurveyID) REFERENCES abundance_surveys(SurveyID),
        UNIQUE(StellarID, Element_Ratio)
    )
    ''')

    
    conn.commit()

In [97]:
def extract_unique_molecules(molecules_str: str) -> list[str]:
    """Extract unique molecules from the comma-separated string."""
    if pd.isna(molecules_str):
        return []
    return [m.strip() for m in molecules_str.split(',')]

In [98]:
def load_data(csv_path: str, db_path: str):
    df = pd.read_csv(csv_path)

    conn = sqlite3.connect(db_path)
    create_database_schema(conn)

    systems_df = df[['hostname', 'sy_snum', 'sy_pnum', 'ra', 'dec']].drop_duplicates()
    systems_df = systems_df.rename(columns={'hostname': 'sys_name'})
    print(systems_df)
    systems_df.to_sql('system', conn, if_exists='append', index=False)
    
    discoveries_df = df[['disc_refname', 'disc_pubdate']].drop_duplicates()
    discoveries_df.to_sql('discovery', conn, if_exists='append', index=False)
    
    planets_df = df[['pl_name', 'hostname', 'mass', 'radius', 'orbital_period', 
                    'tsm', 'disc_refname']].drop_duplicates()
    planets_df = planets_df.rename(columns={'disc_refname': 'disc'})
    planets_df.to_sql('planet', conn, if_exists='append', index=False)
    
    all_molecules = set()
    for molecules in df['molecules'].dropna():
        all_molecules.update(extract_unique_molecules(molecules))
    
    cursor = conn.cursor()
    for molecule in all_molecules:
        cursor.execute('INSERT INTO molecule (name) VALUES (?)', (molecule,))
    conn.commit()
    
    cursor.execute('SELECT id, name FROM molecule')
    molecule_lookup = dict(cursor.fetchall())
    molecule_lookup = {v: k for k, v in molecule_lookup.items()}
    
    
    conn.commit()
    conn.close()

In [99]:
def extract_molecule_set(string):
    # Adjust regex to include keys with special characters (e.g., C/O, spaces, etc.)
    key_value_pairs = re.findall(r'"([\w\s:/.-]+)"\s*:\s*"Detection"', string)
    isolated_keys = set()
    for key in key_value_pairs:
        # Check if "C/O" exists within the key and add it directly
        if "C/O" in key:
            isolated_keys.add("C/O")
        elif "Featureless" not in key:
            isolated_keys.add(key)
    return isolated_keys
df = pd.read_csv('../preprocessed_data/planet_atmosphere.csv')


molecules = set()
planet_has = {}
for i in range(len(df['molecules'])):
    mol_i = extract_molecule_set(df['molecules'][i])
    try:
        planet_has[df['pl_name'][i]] = planet_has[df['pl_name'][i]].union(mol_i)
    except:
        planet_has[df['pl_name'][i]] = mol_i
    molecules = molecules.union(mol_i)


In [100]:
def add_planet_molecules(connection, planet_molecules):
    """
    Add molecules for each planet to the planet_has_molecule table.

    Args:
        connection (sqlite3.Connection): SQLite database connection.
        planet_molecules (dict): Dictionary where keys are planet names (str)
                                  and values are lists of molecule names (str).
    """
    cursor = connection.cursor()
    cursor.execute("SELECT name FROM molecule")
    existing_mols = {row[0] for row in cursor.fetchall()}

    # Iterate over each planet and its molecules
    for planet, molecules in planet_molecules.items():
        # Ensure the planet exists in the planet table
        cursor.execute('''
        INSERT OR IGNORE INTO planet (pl_name) VALUES (?)
        ''', (planet,))
        
        for molecule in molecules:
            if molecule not in existing_mols:

                # Ensure the molecule exists in the molecule table
                cursor.execute('''
                INSERT OR IGNORE INTO molecule (name) VALUES (?)
                ''', (molecule,))
                existing_mols.add(molecule)
            
            # Get the molecule ID
            cursor.execute('''
            SELECT id FROM molecule WHERE name = ?
            ''', (molecule,))
            molecule_id = cursor.fetchone()[0]
            
            # Insert into planet_has_molecule table
            cursor.execute('''
            INSERT OR IGNORE INTO planet_has_molecule (pl_name, molecule_id)
            VALUES (?, ?)
            ''', (planet, molecule_id))

    # Commit the changes
    connection.commit()


def add_planet_instruments(connection, planet_observed):
    """
    Add instruments for each planet to the planet_characterized_by table.

    Args:
        connection (sqlite3.Connection): SQLite database connection.
        planet_observed_by (dict): Dictionary where keys are planet names (str)
                                   and values are lists of instrument names (str).
    """
    cursor = connection.cursor()

    # Fetch all existing instruments from the database
    cursor.execute("SELECT name FROM instrument")
    existing_instruments = {row[0] for row in cursor.fetchall()}  # Set of instrument names

    # Iterate over each planet and its instruments
    for planet, instruments in planet_observed.items():
        # Ensure the planet exists in the planet table
        cursor.execute('''
        INSERT OR IGNORE INTO planet (pl_name) VALUES (?)
        ''', (planet,))
        
        for instrument in instruments:
            # Only insert instrument if it's not already in the existing_instruments set
            if instrument not in existing_instruments:
                cursor.execute('''
                INSERT INTO instrument (name) VALUES (?)
                ''', (instrument,))
                existing_instruments.add(instrument)  # Add the new instrument to the set
            
            # Get the instrument ID
            cursor.execute('''
            SELECT id FROM instrument WHERE name = ?
            ''', (instrument,))
            instrument_id = cursor.fetchone()[0]
            
            # Insert into planet_characterized_by table
            cursor.execute('''
            INSERT OR IGNORE INTO planet_characterized_by (pl_name, instrument_id)
            VALUES (?, ?)
            ''', (planet, instrument_id))

    # Commit the changes
    connection.commit()




In [10]:
import sqlite3
import pandas as pd


connection = sqlite3.connect("../test.db")
create_database_schema(connection)

cursor = connection.cursor()
# Convert the set into a list of tuples
molecule_tuples = [(name,) for name in molecules]

# Insert the molecules into the table
cursor.executemany('''
INSERT OR IGNORE  INTO molecule (name) VALUES (?)
''', molecule_tuples)

# Commit the changes and close the connection
connection.commit()
print(f"{cursor.rowcount} rows inserted.")
connection.close()

0 rows inserted.


In [11]:
connection = sqlite3.connect("../test.db")
cursor = connection.cursor()

# Add planet-molecule mappings
add_planet_molecules(connection, planet_has)

# Query the planet_has_molecule table to verify
cursor.execute('''
SELECT * FROM planet_has_molecule
''')
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the connection
connection.close()


('2M 0103-55 (AB) b', 7)
('2M0437 b', 1)
('2M0437 b', 32)
('2M0437 b', 35)
('51 Eri b', 3)
('51 Peg b', 1)
('51 Peg b', 35)
('55 Cnc b', 7)
('beta Pic b', 1)
('beta Pic b', 3)
('beta Pic b', 35)
('CI Tau b', 35)
('CoRoT-1 b', 1)
('CoRoT-1 b', 9)
('CoRoT-1 b', 23)
('CoRoT-1 b', 34)
('GJ 1132 b', 11)
('GJ 1214 b', 18)
('GJ 3470 b', 1)
('GJ 3470 b', 18)
('GJ 3470 b', 9)
('GJ 3470 b', 32)
('GJ 3470 b', 27)
('GJ 3470 b', 7)
('GJ 3470 b', 35)
('GJ 436 b', 7)
('GJ 9827 d', 1)
('HAT-P-1 b', 1)
('HAT-P-1 b', 33)
('HAT-P-1 b', 40)
('HAT-P-11 b', 1)
('HAT-P-11 b', 17)
('HAT-P-11 b', 18)
('HAT-P-11 b', 7)
('HAT-P-12 b', 1)
('HAT-P-18 b', 1)
('HAT-P-18 b', 18)
('HAT-P-18 b', 9)
('HAT-P-18 b', 26)
('HAT-P-18 b', 15)
('HAT-P-26 b', 1)
('HAT-P-26 b', 18)
('HAT-P-26 b', 38)
('HAT-P-32 b', 1)
('HAT-P-32 b', 18)
('HAT-P-32 b', 9)
('HAT-P-32 b', 15)
('HAT-P-32 b', 7)
('HAT-P-41 b', 1)
('HAT-P-41 b', 31)
('HAT-P-41 b', 38)
('HAT-P-41 b', 5)
('HAT-P-55 b', 33)
('HAT-P-55 b', 5)
('HAT-P-65 b', 38)
('HAT-P-67

In [287]:
instruments = set()
planet_observed_by = {}

for i in range(len(df['observation_type'])):
    input_string = df['observation_type'][i]
    
    # Extract observation type from the string
    match = re.match(r'^"(.*)"$', input_string)
    if match:
        obs_i = match.group(1)  # Extract the matched observation type
        
        # Ensure obs_i is a set for the union operation
        obs_i_set = {obs_i}
        
        # Add observations to the planet_observed_by dictionary
        if df['pl_name'][i] in planet_observed_by:
            planet_observed_by[df['pl_name'][i]] = planet_observed_by[df['pl_name'][i]].union(obs_i_set)
        else:
            planet_observed_by[df['pl_name'][i]] = obs_i_set
        
        # Add to the global instruments set
        instruments = instruments.union(obs_i_set)


In [288]:
import sqlite3

connection = sqlite3.connect("../test.db")
create_database_schema(connection)

cursor = connection.cursor()
# Convert the set into a list of tuples
instrument_tuples = [(name,) for name in instruments]

# Insert the molecules into the table
cursor.executemany('''
INSERT INTO instrument (name) VALUES (?)
''', instrument_tuples)

# Commit the changes and close the connection
connection.commit()
print(f"{cursor.rowcount} rows inserted.")
connection.close()

16 rows inserted.


In [289]:
connection = sqlite3.connect("../test.db")
cursor = connection.cursor()

# Add planet-molecule mappings
add_planet_instruments(connection, planet_observed_by)

# Query the planet_has_molecule table to verify
cursor.execute('''
SELECT * FROM planet_characterized_by
''')
rows = cursor.fetchall()
for row in rows:
    print(row)

# Close the connection
connection.close()


('2M 0103-55 (AB) b', 14)
('2M0437 b', 10)
('51 Eri b', 10)
('51 Peg b', 10)
('51 Peg b', 16)
('51 Peg b', 14)
('55 Cnc b', 12)
('55 Cnc e', 10)
('55 Cnc e', 5)
('55 Cnc e', 6)
('55 Cnc e', 7)
('55 Cnc e', 14)
('55 Cnc e', 9)
('55 Cnc e', 16)
('AU Mic b', 10)
('AU Mic b', 12)
('AU Mic b', 8)
('AU Mic b', 14)
('beta Pic b', 10)
('CI Tau b', 10)
('CoRoT-1 b', 3)
('CoRoT-1 b', 7)
('CoRoT-2 b', 16)
('DS Tuc A b', 14)
('GJ 1132 b', 4)
('GJ 1132 b', 3)
('GJ 1132 b', 6)
('GJ 1132 b', 7)
('GJ 1214 b', 10)
('GJ 1214 b', 6)
('GJ 1214 b', 4)
('GJ 1214 b', 7)
('GJ 1214 b', 3)
('GJ 1214 b', 15)
('GJ 1252 b', 16)
('GJ 3470 b', 10)
('GJ 3470 b', 12)
('GJ 3470 b', 3)
('GJ 3470 b', 9)
('GJ 367 b', 6)
('GJ 436 b', 10)
('GJ 436 b', 12)
('GJ 436 b', 14)
('GJ 486 b', 10)
('GJ 486 b', 4)
('GJ 486 b', 6)
('GJ 9827 b', 10)
('GJ 9827 d', 10)
('GJ 9827 d', 15)
('GJ 9827 d', 7)
('GJ 9827 d', 1)
('HAT-P-1 b', 12)
('HAT-P-1 b', 3)
('HAT-P-1 b', 7)
('HAT-P-10 A b', 10)
('HAT-P-11 b', 10)
('HAT-P-11 b', 12)
('HAT-P-

In [32]:
ids['ID'][10]

'TOI-270'

In [35]:
def add_host_ids(connection, primary_id, alternate_ids,primary_id_column='primary_id', alternate_id_column='alternate_id'):
    cursor = connection.cursor()
    # Iterate over each row in the DataFrame
    if alternate_ids:
        for i in range(len(alternate_ids)):
            alternate_id = alternate_ids['ID'][i]
            # # Ensure the primary_id exists in the system table
            # cursor.execute('''
            # INSERT OR IGNORE INTO system (sys_name) VALUES (?)
            # ''', (primary_id,))
    
            # Insert into the host_ids table
            cursor.execute('''
            INSERT OR IGNORE INTO host_ids (primary_id, alternate_id)
            VALUES (?, ?)
            ''', (primary_id, alternate_id))
    
        # Commit changes to the database
    connection.commit()

In [36]:
df = pd.read_csv('../preprocessed_data/planet_and_system.csv')
hosts = list(set(df['hostname']))
connection = sqlite3.connect("../test.db")
create_database_schema(connection)

for ind in range(len(hosts)):
    ids = Simbad.query_objectids(hosts[ind])
    add_host_ids(connection, hosts[ind], ids)
connection.close()


In [116]:
from astropy.io.votable import parse

# Specify the path to your VOTable file
votable_path = "../original data/brewer_abundance.vot"

# Parse the VOTable file
votable = parse(votable_path)

# Access the first table in the VOTable
table = votable.get_first_table()

original_column_names = [field.name for field in table.fields]


# Convert the table to an Astropy Table for easier manipulation
from astropy.table import Table
astropy_table = Table(table.array)

# Rename the columns in the Astropy Table
for old_name, new_name in zip(astropy_table.colnames, original_column_names):
    astropy_table.rename_column(old_name, new_name)

# Print the updated table with original column names
print(astropy_table)

# Convert to Pandas DataFrame
df = astropy_table.to_pandas()


SPOCS    Name    Teff log(g) [M/H] S/N [C/H] [N/H] [O/H] [Na/H] [Mg/H] [Al/H] [Si/H] [Ca/H] [Ti/H] [V/H] [Cr/H] [Mn/H] ... e_vsini e_[C/H] e_[N/H] e_[O/H] e_[Na/H] e_[Mg/H] e_[Al/H] e_[Si/H] e_[Ca/H] e_[Ti/H] e_[V/H] e_[Cr/H] e_[Mn/H] e_[Fe/H] e_[Ni/H] e_[Y/H] CKS recno
----- ---------- ---- ------ ----- --- ----- ----- ----- ------ ------ ------ ------ ------ ------ ----- ------ ------ ... ------- ------- ------- ------- -------- -------- -------- -------- -------- -------- ------- -------- -------- -------- -------- ------- --- -----
 2281   KOI-3248 5742   4.34 -0.07  50 -0.06 -0.04 -0.05  -0.17  -0.07   0.03  -0.04  -0.06  -0.06 -0.16  -0.06  -0.12 ...     0.5    0.03    0.08    0.06     0.03     0.02     0.04     0.02     0.02     0.02    0.04     0.02     0.03     0.02     0.02    0.04   3     1
 2361   KOI-4273 6123   4.11  0.14  46  0.07 -0.02  0.18   0.14   0.11   0.13   0.08   0.24   0.25 -0.06   0.18   0.01 ...     0.5    0.04    0.08    0.06     0.03     0.02     0.04     0

In [52]:
df['StellarID']

0         KOI-3248
1         KOI-4273
2         KOI-3605
3         KOI-3197
4         KOI-1353
           ...    
1144    BD+52_277A
1145     HD 239643
1146     HD 203802
1147     HD 240051
1148     HD 215714
Name: StellarID, Length: 1149, dtype: object

In [108]:
conn = sqlite3.connect("../test.db")
create_database_schema(connection)
cursor = conn.cursor()

try:
    cursor.execute("INSERT INTO abundance_surveys (Name) VALUES ('Brewer')")
    conn.commit()
except sqlite3.IntegrityError:
    print("Survey already exists.")

try:
    cursor.execute("INSERT INTO abundance_surveys (Name) VALUES ('APOGEE')")
    conn.commit()
except sqlite3.IntegrityError:
    print("Survey already exists.")

Survey already exists.
Survey already exists.


In [115]:
df.columns

Index(['host', 'apogee_id', 'ra_x', 'dec_x', 'ra_y', 'dec_y', 'glon', 'glat',
       'snr', 'extratarg', 'teff', 'teff_err', 'logg', 'logg_err', 'm_h',
       'm_h_err', 'fe_h', 'fe_h_err', 'o_fe', 'o_fe_err', 'na_fe', 'na_fe_err',
       'mg_fe', 'mg_fe_err', 'al_fe', 'al_fe_err', 'k_fe', 'k_fe_err', 'co_fe',
       'co_fe_err'],
      dtype='object')

In [118]:
# Assume 'df' is your DataFrame loaded with Brewer data
df.rename(columns={
    'Name': 'StellarID',
    'Teff': 'Teff',
    'log(g)': 'log_g',
    '[M/H]': 'M_H',
    'S/N': 'SN',
    'Mass': 'Mass',
    'b_Mass': 'l_Mass',
    'B_Mass': 'u_Mass',
    'Age': 'Age',
    'b_Age': 'l_Age',
    'B_Age': 'u_Age',
    'e_[M/H]': 'e_M_H'
}, inplace=True)

# Convert DataFrame to a list of tuples for SQL insertion
stellar_properties = df[['StellarID', 'Teff', 'log_g', 'SN', 'Mass', 'l_Mass', 'u_Mass', 'Age', 'l_Age', 'u_Age', 'M_H', 'e_M_H']]
stellar_properties.to_sql('brewer_stellar_property', con=conn, if_exists='append', index=False)

# # Insert data into brewer_stellar_property
# cursor.executemany('''
# INSERT INTO brewer_stellar_property
# (StellarID, Teff, log_g, SN, Mass, l_Mass, u_Mass, Age, l_Age, u_Age, M_H, e_M_H)
# VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
# ''', stellar_properties)
# conn.commit()

IntegrityError: UNIQUE constraint failed: brewer_stellar_property.StellarID

In [74]:
elements = ['C', 'N', 'O', 'Na', 'Mg', 'Al', 'Si', 'Ca', 'Ti', 'V', 'Cr', 'Mn', 'Fe', 'Ni', 'Y']
survey_id = 1  # Assuming the ID for 'Brewer' is 1 (you might need to verify this)

for element in elements:
    df_element = pd.DataFrame({
        'StellarID': df['StellarID'],
        'Element_Ratio': f'{element}_H',
        'Abundance': df[f'[{element}/H]'],
        'Error': df[f'e_[{element}/H]'],
        'SurveyID': survey_id
    })

    # Convert DataFrame to a list of tuples for SQL insertion

    # Insert data into elemental_abundances table
    df_element.to_sql('elemental_abundances', con=conn, if_exists='append', index=False)

    # cursor.executemany('''
    # INSERT INTO elemental_abundances (StellarID, Element_Ratio, Abundance, Error, SurveyID)
    # VALUES (?, ?, ?, ?, ?)
    # ''', element_records)
    # conn.commit()
conn.close()


In [122]:
df = pd.read_csv("../preprocessed_data/APOGEE_Planet_Hosts.csv")


In [124]:
df.columns

Index(['host', 'apogee_id', 'ra_x', 'dec_x', 'ra_y', 'dec_y', 'glon', 'glat',
       'snr', 'extratarg', 'teff', 'teff_err', 'logg', 'logg_err', 'm_h',
       'm_h_err', 'fe_h', 'fe_h_err', 'o_fe', 'o_fe_err', 'na_fe', 'na_fe_err',
       'mg_fe', 'mg_fe_err', 'al_fe', 'al_fe_err', 'k_fe', 'k_fe_err', 'co_fe',
       'co_fe_err'],
      dtype='object')

In [136]:
conn = sqlite3.connect("../test.db")
create_database_schema(conn)
cursor = conn.cursor()

df = pd.read_csv("../preprocessed_data/APOGEE_Planet_Hosts.csv")
# Assume 'df' is your DataFrame loaded with Brewer data
df.rename(columns={
    'apogee_id': 'StellarID',
    'teff': 'Teff',
    'teff_err': 'e_Teff',
    'logg': 'log_g',
    'logg_err': 'e_log_g',
    'm_h': 'M_H',
    'm_h_err': 'e_M_H',
    'snr': 'SN'
}, inplace=True)

# Convert DataFrame to a list of tuples for SQL insertion
stellar_properties = df[['StellarID', 'Teff', 'e_Teff', 'log_g', 'e_log_g', 'SN', 'M_H', 'e_M_H']]
stellar_properties.to_sql('apogee_stellar_property', con=conn, if_exists='append', index=False)

# # Insert data into brewer_stellar_property
# cursor.executemany('''
# INSERT INTO brewer_stellar_property
# (StellarID, Teff, log_g, SN, Mass, l_Mass, u_Mass, Age, l_Age, u_Age, M_H, e_M_H)
# VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
# ''', stellar_properties)
# conn.commit()

21

In [143]:

# Elements and corresponding column names from the DataFrame
elements_with_columns = {
    'Fe': ('fe_h', 'fe_h_err'),
    'O': ('o_fe', 'o_fe_err'),
    'Na': ('na_fe', 'na_fe_err'),
    'Mg': ('mg_fe', 'mg_fe_err'),
    'Al': ('al_fe', 'al_fe_err'),
    'K': ('k_fe', 'k_fe_err'),
    'Co': ('co_fe', 'co_fe_err')
}

# Assuming SurveyID for APOGEE is 2
survey_id = 2

# Establish database connection
conn = sqlite3.connect("../test.db")

# Iterate over each element and prepare data for insertion
for element, (abundance_col, error_col) in elements_with_columns.items():
    try:
        if element != 'Fe':
        # Create a DataFrame for each element
            df_element = pd.DataFrame({
                'StellarID': df['StellarID'],  # Assuming 'apogee_id' is the unique stellar ID
                'Element_Ratio': f'{element}_Fe',  # Format: '[Element/Fe]'
                'Abundance': df[abundance_col],  # Element abundance column
                'Error': df[error_col],  # Error column
                'SurveyID': survey_id
            })
        else:
            # Create a DataFrame for each element
            df_element = pd.DataFrame({
                'StellarID': df['StellarID'],  # Assuming 'apogee_id' is the unique stellar ID
                'Element_Ratio': f'{element}_H',  # Format: '[Fe/H]'
                'Abundance': df[abundance_col],  # Element abundance column
                'Error': df[error_col],  # Error column
                'SurveyID': survey_id})
        # Insert data into `elemental_abundances` table
        df_element.to_sql('elemental_abundances', con=conn, if_exists='append', index=False)

    except KeyError as e:
        print(f"KeyError for element {element}: {e}")
    except Exception as ex:
        print(f"Error while processing element {element}: {ex}")

# Close the connection
conn.close()

In [105]:
if __name__ == "__main__":
    csv_path = "../preprocessed_data/planet_and_system.csv" 
    db_path = "../test.db"
    load_data(csv_path, db_path)

                     sys_name  sy_snum  sy_pnum          ra        dec
0                         NaN      NaN      NaN         NaN        NaN
1     2MASS J04372171+2651014      1.0      1.0   69.340553  26.850300
2                      51 Eri      3.0      1.0   69.400742  -2.473825
5                      51 Peg      1.0      1.0  344.367540  20.769096
30                     55 Cnc      2.0      5.0  133.146837  28.329815
...                       ...      ...      ...         ...        ...
8174                  WASP-98      2.0      1.0   58.429011 -34.328272
8181              WD 1856+534      3.0      1.0  284.415675  53.509024
8183                     XO-1      1.0      1.0  240.549274  28.169625
8217                     XO-3      1.0      1.0   65.469581  57.817209
8232                     XO-4      1.0      1.0  110.388029  58.268109

[200 rows x 5 columns]


IntegrityError: UNIQUE constraint failed: system.sys_name

In [94]:
systems_df = df[['hostname', 'sy_snum', 'sy_pnum', 'ra', 'dec']].drop_duplicates()
systems_df = systems_df.rename(columns={'hostname': 'sys_name'})
print(systems_df)

                     sys_name  sy_snum  sy_pnum          ra        dec
0                         NaN      NaN      NaN         NaN        NaN
1     2MASS J04372171+2651014      1.0      1.0   69.340553  26.850300
2                      51 Eri      3.0      1.0   69.400742  -2.473825
5                      51 Peg      1.0      1.0  344.367540  20.769096
30                     55 Cnc      2.0      5.0  133.146837  28.329815
...                       ...      ...      ...         ...        ...
8174                  WASP-98      2.0      1.0   58.429011 -34.328272
8181              WD 1856+534      3.0      1.0  284.415675  53.509024
8183                     XO-1      1.0      1.0  240.549274  28.169625
8217                     XO-3      1.0      1.0   65.469581  57.817209
8232                     XO-4      1.0      1.0  110.388029  58.268109

[200 rows x 5 columns]
